<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/5450_Project_SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Research Question

# Business Impact

# Installing New Libraries

In [10]:
!pip install linearmodels

In [11]:
!pip install mplcursors

In [12]:
!pip install ydata_profiling

In [13]:
!pip install catboost

# Importing Libraries

In [14]:
# imports
import pandas as pd
import copy
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import kagglehub
import itertools
import mplcursors
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Reading Data from Kaggle

In [15]:
# Download latest version
path = kagglehub.dataset_download("theworldbank/health-nutrition-and-population-statistics")

print("Path to dataset files:", path)

health_df = pd.read_csv(path + '/data.csv')
orig_health_df = copy.deepcopy(health_df)

100%|██████████| 14.0M/14.0M [00:01<00:00, 11.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/theworldbank/health-nutrition-and-population-statistics/versions/1


In [16]:
health_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Unnamed: 60
0,Arab World,ARB,% of females ages 15-49 having comprehensive c...,SH.HIV.KNOW.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,% of males ages 15-49 having comprehensive cor...,SH.HIV.KNOW.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,133.555013,134.159119,134.857912,134.504576,134.105211,133.569626,...,49.999851,49.887046,49.781207,49.672975,49.536047,49.383745,48.796558,48.196418,NaN,NaN
3,Arab World,ARB,Adults (ages 15+) and children (0-14 years) li...,SH.HIV.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,Adults (ages 15+) and children (ages 0-14) new...,SH.HIV.INCD.TL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Helper Functions

In [17]:
def plot_results(df_test_and_predicted, model):
    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(df_test_and_predicted['Year'], df_test_and_predicted['Actual'], label='True Values') # Access the 'Actual' column data
    plt.plot(df_test_and_predicted['Year'], df_test_and_predicted['Predicted'], label='Predictions', linestyle='dashed') # Access the 'Predicted' column data
    plt.legend(loc='lower center', bbox_to_anchor=(0.5, 1.0), ncol=3)
    plt.title(f'{model} Panel Data Prediction')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.show()

In [18]:
def compare_actual_vs_predicted(X_test, y_test, y_pred, model):
    df_actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    df_actual_vs_predicted['Model'] = model
    df_test_and_predicted = pd.concat([X_test, df_actual_vs_predicted], axis=1)
    df_test_and_predicted['Country Name'] = df_test_and_predicted['Country Code'].apply(lambda x: country_dict.get(x, x))
    return df_test_and_predicted, df_actual_vs_predicted

In [19]:
def plot_kpi(df, x_col, y_col, hue_col):
    title_val = kpi_dict.get(y_col, y_col)
    fig = px.line(df, x=x_col, y=y_col, color=hue_col, title=title_val)
    fig.update_traces(mode="markers+lines", hovertemplate=None)
    fig.update_layout(hovermode="x")
    fig.show()

In [20]:
def show_unique_values(df):
    for e in zip(df.columns, df.dtypes):
        col_name, col_dtype = e
        if col_dtype == 'object':
            unique_values = df[col_name].unique()
            print(f"Unique values for column '{col_name}' are {unique_values}")
            print("---------------------------------------------------------------------")

In [21]:
def show_null_values(df):
    df_len = len(df)
    for col in df.columns:
        total_col_null_values = df[col].isnull().sum()
        pct_null_values = np.round(100 * (total_col_null_values / df_len), 2)
        print(f"Column '{col}' has {pct_null_values}% null values ({total_col_null_values} out of {df_len}).")
        print("---------------------------------------------------------------------")

In [22]:
def show_null_values_per_country(df, ind_col, col, country):
    df_len = len(df)
    total_col_null_values = df[col].isnull().sum()
    pct_null_values = np.round(100 * (total_col_null_values / df_len), 2)
    print(f"For country {country} indicator column '{ind_col}' has {pct_null_values}% null values ({total_col_null_values} out of {df_len}).")
    print("---------------------------------------------------------------------")

In [23]:
def show_zero_values_per_indicator(df, ind_col):
    df_len = len(df)
    total_col_zero_values = df[ind_col].isin([0]).sum()
    pct_zero_values = np.round(100 * (total_col_zero_values / df_len), 2)
    print(f"For indicator column '{ind_col}' has {pct_zero_values}% zero values ({total_col_zero_values} out of {df_len}).")
    print("---------------------------------------------------------------------")
    return pct_zero_values

In [24]:
def show_zero_values_per_ctry_indicator(df, ind_col, ctry):
    df_len = len(df[df['Country Name'] == ctry])
    total_col_zero_values = df[df['Country Name'] == ctry][ind_col].isin([0]).sum()
    pct_zero_values = np.round(100 * (total_col_zero_values / df_len), 2)
    print(f"For country {ctry}, indicator column '{ind_col}' has {pct_zero_values}% zero values ({total_col_zero_values} out of {df_len}).")
    print("---------------------------------------------------------------------")

In [79]:
def impute_data(df, ctry, kpi, fill_method):
    df = df[df['Country Name'] == ctry][[kpi]]
    #df[kpi] = df[kpi].replace(0, np.nan)
    if fill_method == 'knn':
        #scaler = MinMaxScaler()
        #df_scaled = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
        imputer = KNNImputer(n_neighbors=2)
        df[kpi] = imputer.fit_transform(df[[kpi]])
    elif fill_method == 'ffill':
        df.ffill(axis = 0, inplace=True)
    elif fill_method == 'ffill':
        df.ffill(axis = 0, inplace=True)
    elif fill_method == 'bfill':
        df.bfill(axis = 0, inplace=True)
    elif fill_method == 'mean':
        df.fillna(df.mean(), inplace=True)
    return list(itertools.chain(*df.values.tolist()))

# Exploratory Data Analysis

In [26]:
health_df.shape

(89010, 61)

In [27]:
health_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', 'Unnamed: 60'],
      dtype='object')

In [28]:
health_df.dtypes

,0
Country Name,object
Country Code,object
Indicator Name,object
Indicator Code,object
1960,float64
...,...
2012,float64
2013,float64
2014,float64
2015,float64


In [29]:
health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89010 entries, 0 to 89009
Data columns (total 61 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    89010 non-null  object 
 1   Country Code    89010 non-null  object 
 2   Indicator Name  89010 non-null  object 
 3   Indicator Code  89010 non-null  object 
 4   1960            35482 non-null  float64
 5   1961            35325 non-null  float64
 6   1962            35889 non-null  float64
 7   1963            35452 non-null  float64
 8   1964            35483 non-null  float64
 9   1965            35603 non-null  float64
 10  1966            35538 non-null  float64
 11  1967            36022 non-null  float64
 12  1968            35577 non-null  float64
 13  1969            35630 non-null  float64
 14  1970            36910 non-null  float64
 15  1971            37578 non-null  float64
 16  1972            38015 non-null  float64
 17  1973            37542 non-null 

In [30]:
ind_name_list = list(health_df['Indicator Name'].unique())
len(ind_name_list)

345

In [31]:
health_df.groupby(['Country Name']).agg({'Indicator Name' : np.size}).sort_values(by='Indicator Name', ascending=False)

,Indicator Name
Country Name,
Afghanistan,345
Russian Federation,345
Namibia,345
Nauru,345
Nepal,345
...,...
Guam,345
Guatemala,345
Guinea,345


In [32]:
health_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', 'Unnamed: 60'],
      dtype='object')

# Data Processing

In [33]:
kpi_dict = dict(zip(health_df['Indicator Code'], health_df['Indicator Name']))

In [34]:
year_cols = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015']

In [35]:
# Droppinmg unnamed columns
health_df.drop(columns=['Unnamed: 60'], inplace=True)
# replacing NaN with 0
for col in year_cols:
       health_df[col] = health_df[col].fillna(0)

In [36]:
'''
The data set contains both countries and conglomereate of multiple countries.
We are just extracing countries.
'''
country_name_list = ['Afghanistan',
                     'Albania',
                     'Algeria',
                     'American Samoa',
                     'Andorra',
                     'Angola',
                     'Antigua and Barbuda',
                     'Argentina',
                     'Armenia',
                     'Aruba',
                     'Australia',
                     'Austria',
                     'Azerbaijan',
                     'Bahamas, The',
                     'Bahrain',
                     'Bangladesh',
                     'Barbados',
                     'Belarus',
                     'Belgium',
                     'Belize',
                     'Benin',
                     'Bermuda',
                     'Bhutan',
                     'Bolivia',
                     'Bosnia and Herzegovina',
                     'Botswana',
                     'Brazil',
                     'British Virgin Islands',
                     'Brunei Darussalam',
                     'Bulgaria',
                     'Burkina Faso',
                     'Burundi',
                     'Cabo Verde',
                     'Cambodia',
                     'Cameroon',
                     'Canada',
                     'Cayman Islands',
                     'Central African Republic',
                     'Chad',
                     'Channel Islands',
                     'Chile',
                     'China',
                     'Colombia',
                     'Comoros',
                     'Congo, Dem. Rep.',
                     'Congo, Rep.',
                     'Costa Rica',
                     "Cote d'Ivoire",
                     'Croatia',
                     'Cuba',
                     'Curacao',
                     'Cyprus',
                     'Czech Republic',
                     'Denmark',
                     'Djibouti',
                     'Dominica',
                     'Dominican Republic',
                     'Ecuador',
                     'Egypt, Arab Rep.',
                     'El Salvador',
                     'Equatorial Guinea',
                     'Eritrea',
                     'Estonia',
                     'Ethiopia',
                     'Faroe Islands',
                     'Fiji',
                     'Finland',
                     'France',
                     'French Polynesia',
                     'Gabon',
                     'Gambia, The',
                     'Georgia',
                     'Germany',
                     'Ghana',
                     'Gibraltar',
                     'Greece',
                     'Greenland',
                     'Grenada',
                     'Guam',
                     'Guatemala',
                     'Guinea',
                     'Guinea-Bissau',
                     'Guyana',
                     'Haiti',
                     'Honduras',
                     'Hong Kong SAR, China',
                     'Hungary',
                     'Iceland',
                     'India',
                     'Indonesia',
                     'Iran, Islamic Rep.',
                     'Iraq',
                     'Ireland',
                     'Isle of Man',
                     'Israel',
                     'Italy',
                     'Jamaica',
                     'Japan',
                     'Jordan',
                     'Kazakhstan',
                     'Kenya',
                     'Kiribati',
                     'Korea, Dem. People’s Rep.',
                     'Korea, Rep.',
                     'Kosovo',
                     'Kuwait',
                     'Kyrgyz Republic',
                     'Lao PDR',
                     'Latvia',
                     'Lebanon',
                     'Lesotho',
                     'Liberia',
                     'Libya',
                     'Liechtenstein',
                     'Lithuania',
                     'Luxembourg',
                     'Macao SAR, China',
                     'Macedonia, FYR',
                     'Madagascar',
                     'Malawi',
                     'Malaysia',
                     'Maldives',
                     'Mali',
                     'Malta',
                     'Marshall Islands',
                     'Mauritania',
                     'Mauritius',
                     'Mexico',
                     'Micronesia, Fed. Sts.',
                     'Moldova',
                     'Monaco',
                     'Mongolia',
                     'Montenegro',
                     'Morocco',
                     'Mozambique',
                     'Myanmar',
                     'Namibia',
                     'Nauru',
                     'Nepal',
                     'Netherlands',
                     'New Caledonia',
                     'New Zealand',
                     'Nicaragua',
                     'Niger',
                     'Nigeria',
                     'Northern Mariana Islands',
                     'Norway',
                     'Oman',
                     'Pakistan',
                     'Palau',
                     'Panama',
                     'Papua New Guinea',
                     'Paraguay',
                     'Peru',
                     'Philippines',
                     'Poland',
                     'Portugal',
                     'Puerto Rico',
                     'Qatar',
                     'Romania',
                     'Russian Federation',
                     'Rwanda',
                     'Samoa',
                     'San Marino',
                     'Sao Tome and Principe',
                     'Saudi Arabia',
                     'Senegal',
                     'Serbia',
                     'Seychelles',
                     'Sierra Leone',
                     'Singapore',
                     'Sint Maarten (Dutch part)',
                     'Slovak Republic',
                     'Slovenia',
                     'Solomon Islands',
                     'Somalia',
                     'South Africa',
                     'South Sudan',
                     'Spain',
                     'Sri Lanka',
                     'St. Kitts and Nevis',
                     'St. Lucia',
                     'St. Martin (French part)',
                     'St. Vincent and the Grenadines',
                     'Sudan',
                     'Suriname',
                     'Swaziland',
                     'Sweden',
                     'Switzerland',
                     'Syrian Arab Republic',
                     'Tajikistan',
                     'Tanzania',
                     'Thailand',
                     'Timor-Leste',
                     'Togo',
                     'Tonga',
                     'Trinidad and Tobago',
                     'Tunisia',
                     'Turkey',
                     'Turkmenistan',
                     'Turks and Caicos Islands',
                     'Tuvalu',
                     'Uganda',
                     'Ukraine',
                     'United Arab Emirates',
                     'United Kingdom',
                     'United States',
                     'Uruguay',
                     'Uzbekistan',
                     'Vanuatu',
                     'Venezuela, RB',
                     'Vietnam',
                     'Virgin Islands (U.S.)',
                     'West Bank and Gaza',
                     'Yemen, Rep.',
                     'Zambia',
                     'Zimbabwe'
                     ]

In [37]:
country_year_health_df = health_df[health_df['Country Name'].isin(country_name_list)].reset_index(drop=True)
country_year_health_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afghanistan,AFG,% of females ages 15-49 having comprehensive c...,SH.HIV.KNOW.FE.ZS,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0000,0.00,0.0000,0.0000,0.0000,1.5000,0.000,0.0000,0.0000,0.0
1,Afghanistan,AFG,% of males ages 15-49 having comprehensive cor...,SH.HIV.KNOW.MA.ZS,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0
2,Afghanistan,AFG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,145.321,145.321,145.321,145.321,145.321,145.321,...,123.1616,117.39,111.4708,105.5516,99.6324,93.7132,87.794,82.2638,76.7336,0.0
3,Afghanistan,AFG,Adults (ages 15+) and children (0-14 years) li...,SH.HIV.TOTL,0.000,0.000,0.000,0.000,0.000,0.000,...,3300.0000,3500.00,3800.0000,4200.0000,4500.0000,4900.0000,5300.000,5900.0000,6400.0000,6900.0
4,Afghanistan,AFG,Adults (ages 15+) and children (ages 0-14) new...,SH.HIV.INCD.TL,0.000,0.000,0.000,0.000,0.000,0.000,...,500.0000,1000.00,1000.0000,1000.0000,1000.0000,1000.0000,1000.000,1000.0000,1000.0000,1000.0


In [38]:
#health_df_1 = country_year_health_df.set_index(['Country Name', 'Indicator Code'])[year_cols].unstack()
#health_df_1

In [39]:
'''
Formatting the dataframe structure.
In the original dataframe, the data is laid out for each country-kpi, data is
present in each row across the years.
In the new format, we are placing all the kpis for a country in a single row.
There will be multiple rows for each country pertaining to each year
ranging from 1960 thru 2015.
'''
comb_list = [list(country_year_health_df['Country Name'].unique())] + [year_cols]
result = list(itertools.product(*comb_list))
df_x = pd.DataFrame(result, columns=['Country Name', 'Year'])
for col in country_year_health_df['Indicator Code'].unique():
    df_x[col] = 0
df_x.set_index(['Country Name', 'Year'], inplace=True)
df_x.reset_index(inplace=True)
col_list = list(df_x.columns)

# Create an empty list to store the DataFrames
all_dfs = []
for ctry in country_year_health_df['Country Name'].unique():
    temp_df = country_year_health_df[country_year_health_df['Country Name'] == ctry].iloc[:, 3:].T
    series_names = temp_df.iloc[0, :]
    temp_df = temp_df.iloc[1:, :]
    temp_df.columns = series_names
    temp_df['Country Name'] = ctry
    temp_df.reset_index(inplace=True)
    temp_df.rename(columns={'index' : 'Year'}, inplace=True)
    temp_df = temp_df[col_list]

    # Append the temp_df to the list instead of using append
    all_dfs.append(temp_df)

# Concatenate all DataFrames in the list outside the loop
df_x = pd.concat(all_dfs, ignore_index=True)

<ipython-input-39-3a7555b5af9e>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_x[col] = 0
<ipython-input-39-3a7555b5af9e>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_x[col] = 0
<ipython-input-39-3a7555b5af9e>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_x[col] = 0
<ipython-inpu

In [40]:
df_x.head(50)

Indicator Code,Country Name,Year,SH.HIV.KNOW.FE.ZS,SH.HIV.KNOW.MA.ZS,SP.ADO.TFRT,SH.HIV.TOTL,SH.HIV.INCD.TL,SH.DYN.AIDS,SH.HIV.INCD,SP.DYN.SMAM.FE,...,SP.URB.TOTL,SP.URB.TOTL.IN.ZS,SP.URB.GROW,SI.POV.URHC,SH.MLR.PREG.ZS,SH.MLR.NETS.ZS,SH.MLR.SPF2.ZS,SN.ITK.VITA.ZS,SP.DYN.WFRT,SH.DYN.AIDS.FE.ZS
0,Afghanistan,1960,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,739462.0,8.221,5.253807,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,1961,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,779754.0,8.508,5.305559,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,1962,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,822719.0,8.805,5.363622,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,1963,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,868325.0,9.11,5.395136,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,1964,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,917022.0,9.426,5.456539,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Afghanistan,1965,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,968697.0,9.75,5.482041,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Afghanistan,1966,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,1023511.0,10.085,5.504228,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Afghanistan,1967,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,1081445.0,10.43,5.505924,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Afghanistan,1968,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,1143293.0,10.786,5.561459,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Afghanistan,1969,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,1209937.0,11.152,5.66556,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df_x.shape

(12152, 347)

In [42]:
series_names = df_x.columns[2:]

In [43]:
# Checking % of data populated per country.
for series in series_names:
    print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))

<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Co

SH.HIV.KNOW.FE.ZS Indicator Code        SH.HIV.KNOW.FE.ZS
Country Name                           
Afghanistan                         1.5
Albania                            34.0
Algeria                            21.4
American Samoa                      0.0
Andorra                             0.0
...                                 ...
Virgin Islands (U.S.)               0.0
West Bank and Gaza                 15.6
Yemen, Rep.                         1.3
Zambia                            113.4
Zimbabwe                          217.8

[217 rows x 1 columns]
SH.HIV.KNOW.MA.ZS Indicator Code        SH.HIV.KNOW.MA.ZS
Country Name                           
Afghanistan                         0.0
Albania                            20.3
Algeria                             0.0
American Samoa                      0.0
Andorra                             0.0
...                                 ...
Virgin Islands (U.S.)               0.0
West Bank and Gaza                  0.0
Yemen, Rep.         

<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Co

 Indicator Code        SP.POP.DPND.OL
Country Name                        
Afghanistan                263.06331
Albania                     619.0265
Algeria                   399.697735
American Samoa                   0.0
Andorra                          0.0
...                              ...
Virgin Islands (U.S.)     650.217239
West Bank and Gaza        122.000892
Yemen, Rep.               318.779573
Zambia                     301.87999
Zimbabwe                  343.593086

[217 rows x 1 columns]
SP.POP.DPND.YG Indicator Code        SP.POP.DPND.YG
Country Name                        
Afghanistan              5052.924595
Albania                  3144.396091
Algeria                  4175.822876
American Samoa                   0.0
Andorra                          0.0
...                              ...
Virgin Islands (U.S.)     2919.86445
West Bank and Gaza       2309.317075
Yemen, Rep.              5238.679638
Zambia                   5242.578892
Zimbabwe                 5009.04826

<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Co

...

Indicator Code        SH.TBS.CURE.ZS
Country Name                        
Afghanistan                   1380.0
Albania                       1128.0
Algeria                       1659.0
American Samoa                1208.0
Andorra                       1400.0
...                              ...
Virgin Islands (U.S.)           50.0
West Bank and Gaza            1129.0
Yemen, Rep.                   1521.0
Zambia                        1293.0
Zimbabwe                      1343.0

[217 rows x 1 columns]
SL.UEM.TOTL.FE.ZS Indicator Code        SL.UEM.TOTL.FE.ZS
Country Name                           
Afghanistan                  380.899996
Albania                      397.999999
Algeria                      837.099993
American Samoa                      0.0
Andorra                             0.0
...                                 ...
Virgin Islands (U.S.)               0.0
West Bank and Gaza           427.599998
Yemen, Rep.                  864.900003
Zambia                            311

<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Co

Indicator Code        SP.URB.TOTL.IN.ZS
Country Name                           
Afghanistan                     976.684
Albania                        2165.585
Algeria                        2845.731
American Samoa                 4429.608
Andorra                        4855.862
...                                 ...
Virgin Islands (U.S.)          4615.275
West Bank and Gaza             1876.621
Yemen, Rep.                    1158.764
Zambia                         1949.701
Zimbabwe                       1443.368

[217 rows x 1 columns]
SP.URB.GROW Indicator Code        SP.URB.GROW
Country Name                     
Afghanistan            251.603326
Albania                126.513583
Algeria                216.732134
American Samoa          131.69409
Andorra                213.975779
...                           ...
Virgin Islands (U.S.)  172.791238
West Bank and Gaza      91.003613
Yemen, Rep.            304.383139
Zambia                 255.754712
Zimbabwe               241.711837

[

<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Country Name']).agg({series : np.sum}))
<ipython-input-43-894550dfc28e>:3: FutureWarning: The provided callable <function sum at 0x7bbb279fc670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  print(series, df_x.groupby(['Co

SN.ITK.VITA.ZS Indicator Code        SN.ITK.VITA.ZS
Country Name                        
Afghanistan                   1317.0
Albania                          0.0
Algeria                          0.0
American Samoa                   0.0
Andorra                          0.0
...                              ...
Virgin Islands (U.S.)            0.0
West Bank and Gaza               0.0
Yemen, Rep.                    307.0
Zambia                         998.0
Zimbabwe                       570.0

[217 rows x 1 columns]
SP.DYN.WFRT Indicator Code        SP.DYN.WFRT
Country Name                     
Afghanistan                   0.0
Albania                       3.8
Algeria                       0.0
American Samoa                0.0
Andorra                       0.0
...                           ...
Virgin Islands (U.S.)         0.0
West Bank and Gaza            0.0
Yemen, Rep.                  13.7
Zambia                       25.2
Zimbabwe                     18.1

[217 rows x 1 columns]
SH

In [44]:
# Checking % of zero values per country-kpi level.
for ctry in country_year_health_df['Country Name'].unique():
    for series in series_names:
        show_zero_values_per_ctry_indicator(df_x, series, ctry)

Streaming output truncated to the last 5000 lines.
For country Vanuatu, indicator column 'SP.POP.80UP.MA.5Y' has 0.0% zero values (0 out of 56).
---------------------------------------------------------------------
For country Vanuatu, indicator column 'SP.POP.1014.MA.5Y' has 0.0% zero values (0 out of 56).
---------------------------------------------------------------------
For country Vanuatu, indicator column 'SP.POP.GROW' has 0.0% zero values (0 out of 56).
---------------------------------------------------------------------
For country Vanuatu, indicator column 'SP.POP.TOTL.FE.IN' has 0.0% zero values (0 out of 56).
---------------------------------------------------------------------
For country Vanuatu, indicator column 'SP.POP.TOTL.FE.ZS' has 0.0% zero values (0 out of 56).
---------------------------------------------------------------------
For country Vanuatu, indicator column 'SP.POP.TOTL.MA.IN' has 0.0% zero values (0 out of 56).
-----------------------------------------

In [45]:
# Checking % of zero values at kpi level across all countries.
for series in series_names:
    show_zero_values_per_indicator(df_x, series)

For indicator column 'SH.HIV.KNOW.FE.ZS' has 97.82% zero values (11887 out of 12152).
---------------------------------------------------------------------
For indicator column 'SH.HIV.KNOW.MA.ZS' has 98.8% zero values (12006 out of 12152).
---------------------------------------------------------------------
For indicator column 'SP.ADO.TFRT' has 12.2% zero values (1482 out of 12152).
---------------------------------------------------------------------
For indicator column 'SH.HIV.TOTL' has 77.11% zero values (9370 out of 12152).
---------------------------------------------------------------------
For indicator column 'SH.HIV.INCD.TL' has 77.11% zero values (9370 out of 12152).
---------------------------------------------------------------------
For indicator column 'SH.DYN.AIDS' has 77.11% zero values (9370 out of 12152).
---------------------------------------------------------------------
For indicator column 'SH.HIV.INCD' has 77.11% zero values (9370 out of 12152).
------------

In [46]:
list(range(int(min(year_cols)), int(max(year_cols)), 5))

[1960, 1965, 1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010]

In [47]:
# Checking % of zero values for each kpi across countries with different year cut off.
# The reason for using year cut off is that not alway the kpi values are populated from certain years.
# This % values will help us to use the ones that have zero values under certain threshold.
kpi_matrix = []
for series in series_names:
    for year in list(range(int(min(year_cols)), int(max(year_cols)), 5)):
        print(f"Cut off year {year}")
        pct_zero_values = show_zero_values_per_indicator(df_x[df_x['Year'] >= str(year)], series)
        kpi_matrix.append([year, series, pct_zero_values])
df_kpi = pd.DataFrame(kpi_matrix, columns=['Year', 'Indicator', 'Pct_Zero_Values'])
df_kpi.head()

Streaming output truncated to the last 5000 lines.
For indicator column 'SH.DYN.MORT.MA' has 88.52% zero values (5955 out of 6727).
---------------------------------------------------------------------
Cut off year 1990
For indicator column 'SH.DYN.MORT.MA' has 86.32% zero values (4870 out of 5642).
---------------------------------------------------------------------
Cut off year 1995
For indicator column 'SH.DYN.MORT.MA' has 87.29% zero values (3978 out of 4557).
---------------------------------------------------------------------
Cut off year 2000
For indicator column 'SH.DYN.MORT.MA' has 83.32% zero values (2893 out of 3472).
---------------------------------------------------------------------
Cut off year 2005
For indicator column 'SH.DYN.MORT.MA' has 83.83% zero values (2001 out of 2387).
---------------------------------------------------------------------
Cut off year 2010
For indicator column 'SH.DYN.MORT.MA' has 70.35% zero values (916 out of 1302).
------------------------

,Year,Indicator,Pct_Zero_Values
0,1960,SH.HIV.KNOW.FE.ZS,97.82
1,1965,SH.HIV.KNOW.FE.ZS,97.61
2,1970,SH.HIV.KNOW.FE.ZS,97.35
3,1975,SH.HIV.KNOW.FE.ZS,97.02
4,1980,SH.HIV.KNOW.FE.ZS,96.61


In [48]:
df_kpi[df_kpi['Indicator'] == 'SH.HIV.KNOW.FE.ZS']

,Year,Indicator,Pct_Zero_Values
0,1960,SH.HIV.KNOW.FE.ZS,97.82
1,1965,SH.HIV.KNOW.FE.ZS,97.61
2,1970,SH.HIV.KNOW.FE.ZS,97.35
3,1975,SH.HIV.KNOW.FE.ZS,97.02
4,1980,SH.HIV.KNOW.FE.ZS,96.61
5,1985,SH.HIV.KNOW.FE.ZS,96.06
6,1990,SH.HIV.KNOW.FE.ZS,95.30
7,1995,SH.HIV.KNOW.FE.ZS,94.18
8,2000,SH.HIV.KNOW.FE.ZS,92.45
9,2005,SH.HIV.KNOW.FE.ZS,91.24


In [49]:
df_kpi[df_kpi['Indicator'] == 'SH.TBS.PREV']

,Year,Indicator,Pct_Zero_Values
3179,1960,SH.TBS.PREV,57.68
3180,1965,SH.TBS.PREV,53.53
3181,1970,SH.TBS.PREV,48.48
3182,1975,SH.TBS.PREV,42.19
3183,1980,SH.TBS.PREV,34.17
3184,1985,SH.TBS.PREV,23.55
3185,1990,SH.TBS.PREV,8.84
3186,1995,SH.TBS.PREV,9.48
3187,2000,SH.TBS.PREV,10.57
3188,2005,SH.TBS.PREV,12.78


In [50]:
df_kpi_agg = df_kpi.groupby(['Indicator']).agg({'Pct_Zero_Values' : np.mean}).reset_index().sort_values(by='Pct_Zero_Values', ascending=True)
df_kpi_agg[df_kpi_agg['Pct_Zero_Values'] < 20]

<ipython-input-50-1ce624dd835f>:1: FutureWarning: The provided callable <function mean at 0x7bbb279fd5a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_kpi_agg = df_kpi.groupby(['Indicator']).agg({'Pct_Zero_Values' : np.mean}).reset_index().sort_values(by='Pct_Zero_Values', ascending=True)


,Indicator,Pct_Zero_Values
329,SP.POP.TOTL,0.467273
328,SP.POP.GROW,0.589091
343,SP.URB.TOTL.IN.ZS,1.203636
342,SP.URB.TOTL,1.390000
341,SP.URB.GROW,1.442727
...,...,...
165,SP.ADO.TFRT,15.068182
181,SP.DYN.TO65.FE.ZS,15.068182
182,SP.DYN.TO65.MA.ZS,15.068182
166,SP.DYN.AMRT.FE,17.246364


In [51]:
df_kpi[df_kpi['Indicator'] == 'SP.DYN.AMRT.MA']

,Year,Indicator,Pct_Zero_Values
2046,1960,SP.DYN.AMRT.MA,13.68
2047,1965,SP.DYN.AMRT.MA,13.81
2048,1970,SP.DYN.AMRT.MA,13.96
2049,1975,SP.DYN.AMRT.MA,14.15
2050,1980,SP.DYN.AMRT.MA,14.39
2051,1985,SP.DYN.AMRT.MA,14.78
2052,1990,SP.DYN.AMRT.MA,15.40
2053,1995,SP.DYN.AMRT.MA,16.59
2054,2000,SP.DYN.AMRT.MA,18.52
2055,2005,SP.DYN.AMRT.MA,22.33


In [52]:
df_x.to_csv('health_data_processed.csv', index=False)
df_kpi.to_csv('kpi_matrix.csv', index=False)
df_kpi_agg.to_csv('kpi_matrix_agg.csv', index=False)

In [53]:
df_x[(df_x['Country Name'] == 'Albania')][['Year', 'SH.DYN.AIDS.FE.ZS']]

Indicator Code,Year,SH.DYN.AIDS.FE.ZS
56,1960,0.0
57,1961,0.0
58,1962,0.0
59,1963,0.0
60,1964,0.0
61,1965,0.0
62,1966,0.0
63,1967,0.0
64,1968,0.0
65,1969,0.0


In [54]:
'''
profile = ProfileReport(df_x, title="Profiling Report")
profile
'''

'\nprofile = ProfileReport(df_x, title="Profiling Report")\nprofile\n'

In [55]:
df_x.head(50)

Indicator Code,Country Name,Year,SH.HIV.KNOW.FE.ZS,SH.HIV.KNOW.MA.ZS,SP.ADO.TFRT,SH.HIV.TOTL,SH.HIV.INCD.TL,SH.DYN.AIDS,SH.HIV.INCD,SP.DYN.SMAM.FE,...,SP.URB.TOTL,SP.URB.TOTL.IN.ZS,SP.URB.GROW,SI.POV.URHC,SH.MLR.PREG.ZS,SH.MLR.NETS.ZS,SH.MLR.SPF2.ZS,SN.ITK.VITA.ZS,SP.DYN.WFRT,SH.DYN.AIDS.FE.ZS
0,Afghanistan,1960,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,739462.0,8.221,5.253807,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,1961,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,779754.0,8.508,5.305559,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,1962,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,822719.0,8.805,5.363622,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,1963,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,868325.0,9.11,5.395136,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,1964,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,917022.0,9.426,5.456539,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Afghanistan,1965,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,968697.0,9.75,5.482041,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Afghanistan,1966,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,1023511.0,10.085,5.504228,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Afghanistan,1967,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,1081445.0,10.43,5.505924,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Afghanistan,1968,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,1143293.0,10.786,5.561459,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Afghanistan,1969,0.0,0.0,145.321,0.0,0.0,0.0,0.0,0.0,...,1209937.0,11.152,5.66556,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Imputations

In [56]:
cutoff_pct = 20

In [57]:
selected_kpi_list = df_kpi_agg[df_kpi_agg['Pct_Zero_Values'] <= cutoff_pct]['Indicator'].unique().tolist()
selected_kpi_list

['SP.POP.TOTL',
 'SP.POP.GROW',
 'SP.URB.TOTL.IN.ZS',
 'SP.URB.TOTL',
 'SP.URB.GROW',
 'SP.RUR.TOTL.ZS',
 'SP.RUR.TOTL',
 'SP.RUR.TOTL.ZG',
 'SP.POP.AG13.MA.IN',
 'SP.POP.AG13.FE.IN',
 'SP.POP.AG12.MA.IN',
 'SP.POP.AG12.FE.IN',
 'SP.POP.AG11.MA.IN',
 'SP.POP.AG22.FE.IN',
 'SP.POP.AG11.FE.IN',
 'SP.POP.AG10.MA.IN',
 'SP.POP.AG10.FE.IN',
 'SP.POP.AG09.MA.IN',
 'SP.POP.AG19.MA.IN',
 'SP.POP.AG14.FE.IN',
 'SP.POP.AG25.MA.IN',
 'SP.POP.AG15.FE.IN',
 'SP.POP.AG08.MA.IN',
 'SP.POP.AG20.FE.IN',
 'SP.POP.AG20.MA.IN',
 'SP.POP.AG21.FE.IN',
 'SP.POP.AG18.MA.IN',
 'SP.POP.AG21.MA.IN',
 'SP.POP.AG22.MA.IN',
 'SP.POP.AG23.FE.IN',
 'SP.POP.AG23.MA.IN',
 'SP.POP.AG24.FE.IN',
 'SP.POP.AG24.MA.IN',
 'SP.POP.AG25.FE.IN',
 'SP.POP.AG18.FE.IN',
 'SP.POP.AG17.MA.IN',
 'SP.POP.AG17.FE.IN',
 'SP.POP.AG16.MA.IN',
 'SP.POP.AG16.FE.IN',
 'SP.POP.AG15.MA.IN',
 'SP.POP.AG14.MA.IN',
 'SP.POP.AG08.FE.IN',
 'SP.POP.AG09.FE.IN',
 'SP.POP.AG07.FE.IN',
 'SP.POP.AG01.MA.IN',
 'SP.POP.AG01.FE.IN',
 'SP.POP.AG00.MA.IN',
 '

In [58]:
df_kpi[df_kpi['Indicator'].isin(selected_kpi_list)]

,Year,Indicator,Pct_Zero_Values
22,1960,SP.ADO.TFRT,12.20
23,1965,SP.ADO.TFRT,12.35
24,1970,SP.ADO.TFRT,12.54
25,1975,SP.ADO.TFRT,12.78
26,1980,SP.ADO.TFRT,13.08
...,...,...,...
3713,1990,SP.URB.GROW,1.26
3714,1995,SP.URB.GROW,1.14
3715,2000,SP.URB.GROW,1.07
3716,2005,SP.URB.GROW,1.13


In [59]:
min_pct_zero_values_list = df_kpi[df_kpi['Indicator'].isin(selected_kpi_list)].groupby(['Indicator']).agg({'Pct_Zero_Values' : np.min}).reset_index()['Pct_Zero_Values'].unique().tolist()
min_pct_zero_values_list

<ipython-input-59-be359f86b798>:1: FutureWarning: The provided callable <function min at 0x7bbb279fcdc0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  min_pct_zero_values_list = df_kpi[df_kpi['Indicator'].isin(selected_kpi_list)].groupby(['Indicator']).agg({'Pct_Zero_Values' : np.min}).reset_index()['Pct_Zero_Values'].unique().tolist()


[11.1,
 11.06,
 12.2,
 13.68,
 9.49,
 9.87,
 11.41,
 11.35,
 10.69,
 10.67,
 10.64,
 10.63,
 10.45,
 0.25,
 0.12,
 5.02,
 5.05,
 4.96,
 1.07,
 1.04,
 0.99]

In [60]:
df_kpi_1 = df_kpi[(df_kpi['Indicator'].isin(selected_kpi_list)) & (df_kpi['Pct_Zero_Values'].isin(min_pct_zero_values_list))].sort_values(by='Indicator')
df_kpi_2 = df_kpi_1.groupby(['Indicator']).agg({'Year' : np.min}).reset_index()
df_kpi_2['Year'].value_counts()

<ipython-input-60-4d256c78af38>:2: FutureWarning: The provided callable <function min at 0x7bbb279fcdc0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  df_kpi_2 = df_kpi_1.groupby(['Indicator']).agg({'Year' : np.min}).reset_index()


,count
Year,
2010,52
1990,50
1995,34
1965,14
1960,6
1980,4
2000,3
1975,2
1985,2


In [61]:
year_cutoff = 1990

In [62]:
df_x.isna().any()

,0
Indicator Code,
Country Name,False
Year,False
SH.HIV.KNOW.FE.ZS,False
SH.HIV.KNOW.MA.ZS,False
SP.ADO.TFRT,False
...,...
SH.MLR.NETS.ZS,False
SH.MLR.SPF2.ZS,False
SN.ITK.VITA.ZS,False


In [63]:
col_list = ['Country Name', 'Year'] + selected_kpi_list
df_x_1 = df_x[(df_x['Year'] >= str(year_cutoff))][col_list].reset_index(drop=True)
df_x_1.shape

(5642, 170)

In [64]:
df_x_1.isna().any()

,0
Indicator Code,
Country Name,False
Year,False
SP.POP.TOTL,False
SP.POP.GROW,False
SP.URB.TOTL.IN.ZS,False
...,...
SP.ADO.TFRT,False
SP.DYN.TO65.FE.ZS,False
SP.DYN.TO65.MA.ZS,False


In [65]:
(df_x_1 == 0).any().any()

True

In [66]:
for kpi in selected_kpi_list:
    df_x_1[kpi] = df_x_1[kpi].replace(0, np.nan)

<ipython-input-66-cfca7a86d307>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_x_1[kpi] = df_x_1[kpi].replace(0, np.nan)


In [67]:
df_x_1.isna().any()

,0
Indicator Code,
Country Name,False
Year,False
SP.POP.TOTL,True
SP.POP.GROW,True
SP.URB.TOTL.IN.ZS,True
...,...
SP.ADO.TFRT,True
SP.DYN.TO65.FE.ZS,True
SP.DYN.TO65.MA.ZS,True


In [68]:
df_x_1.head()

Indicator Code,Country Name,Year,SP.POP.TOTL,SP.POP.GROW,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.RUR.TOTL.ZS,SP.RUR.TOTL,SP.RUR.TOTL.ZG,...,SP.DYN.LE00.FE.IN,SP.DYN.IMRT.IN,SH.DYN.MORT,SH.DTH.MORT,SH.DTH.IMRT,SP.ADO.TFRT,SP.DYN.TO65.FE.ZS,SP.DYN.TO65.MA.ZS,SP.DYN.AMRT.FE,SP.DYN.AMRT.MA
0,Afghanistan,1990,12067570.0,3.879694,18.316,2210296.0,5.409140,81.684,9857274.0,3.539933,...,50.867,122.5,181.0,100437.0,68718.0,162.7380,44.99139,39.82877,348.006,396.086
1,Afghanistan,1991,12789374.0,5.809298,18.597,2378440.0,7.331837,81.403,10410934.0,5.464694,...,51.671,118.3,174.2,101417.0,69482.0,163.3270,46.17192,40.92956,339.753,387.388
2,Afghanistan,1992,13745630.0,7.210629,18.882,2595450.0,8.731510,81.118,11150180.0,6.859904,...,52.416,114.4,167.8,104899.0,72376.0,163.9160,47.35245,42.03034,331.500,378.690
3,Afghanistan,1993,14824371.0,7.555156,19.170,2841832.0,9.068900,80.830,11982539.0,7.199487,...,53.098,110.9,162.0,109625.0,76190.0,164.1812,48.09906,42.72012,326.244,373.408
4,Afghanistan,1994,15869967.0,6.815594,19.462,3088613.0,8.327321,80.538,12781354.0,6.453688,...,53.713,107.7,156.8,113758.0,79446.0,164.4464,48.84567,43.40989,320.988,368.125


In [69]:
df_x_1.to_csv('health_data_processed_1.csv', index=False)

In [70]:
df_x_1[selected_kpi_list].isna().sum().sum()

98550

In [71]:
df_x_1[selected_kpi_list].isna().sum()

,0
Indicator Code,
SP.POP.TOTL,15
SP.POP.GROW,25
SP.URB.TOTL.IN.ZS,56
SP.URB.TOTL,67
SP.URB.GROW,71
...,...
SP.ADO.TFRT,792
SP.DYN.TO65.FE.ZS,792
SP.DYN.TO65.MA.ZS,792


In [72]:
df_x_1[['Country Name', 'Year', 'SP.DYN.AMRT.MA']]

Indicator Code,Country Name,Year,SP.DYN.AMRT.MA
0,Afghanistan,1990,396.086
1,Afghanistan,1991,387.388
2,Afghanistan,1992,378.690
3,Afghanistan,1993,373.408
4,Afghanistan,1994,368.125
...,...,...,...
5637,Zimbabwe,2011,517.349
5638,Zimbabwe,2012,477.223
5639,Zimbabwe,2013,445.135
5640,Zimbabwe,2014,413.047


In [ ]:
country_kpi_pop_list = []
for ctry in df_x_1['Country Name'].unique():
    missing_kpi_count = 0
    for kpi in selected_kpi_list:
        # No value is present for the country-kpi combination.
        # In that case, we are replacing with avg kpi value across all countries.
        if df_x_1[df_x_1['Country Name'] == ctry][kpi].isna().sum() == len(df_x_1[df_x_1['Country Name'] == ctry][kpi]):
            missing_kpi_count += 1
            df_x_1.loc[df_x_1['Country Name'] == ctry, kpi] = df_x_1[kpi].mean(axis=0)
        if df_x_1[df_x_1['Country Name'] == ctry][kpi].isna().sum() > 0 and len(df_x_1[df_x_1['Country Name'] == ctry][kpi]) != df_x_1[df_x_1['Country Name'] == ctry][kpi].isna().sum():
            df_x_1.loc[df_x_1['Country Name'] == ctry, kpi] = impute_data(df_x_1, ctry, kpi, 'knn')
            '''
            if df_x_1[selected_kpi_list].isna().sum().sum() > 0:
                df_x_1.loc[df_x_1['Country Name'] == ctry, kpi] = impute_data(df_x_1, ctry, kpi, 'ffill')
            elif df_x_1[selected_kpi_list].isna().sum().sum() > 0:
                df_x_1.loc[df_x_1['Country Name'] == ctry, kpi] = impute_data(df_x_1, ctry, kpi, 'bfill')
            elif df_x_1[selected_kpi_list].isna().sum().sum() > 0:
                df_x_1.loc[df_x_1['Country Name'] == ctry, kpi] = impute_data(df_x_1, ctry, kpi, 'mean')
            '''
    country_kpi_pop_list.append((ctry, missing_kpi_count, len(df_x_1[df_x_1['Country Name'] == ctry])))

<ipython-input-79-2db7d6f7eaf7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[kpi] = imputer.fit_transform(df[[kpi]])
<ipython-input-79-2db7d6f7eaf7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[kpi] = imputer.fit_transform(df[[kpi]])
<ipython-input-79-2db7d6f7eaf7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
df_country_kpi_pop = pd.DataFrame(country_kpi_pop_list, columns=['Country Name', 'Missing KPI Count', 'Total Records'])
df_country_kpi_pop

In [ ]:
df_country_kpi_pop[df_country_kpi_pop['Missing KPI Count'] > 0].sort_values(by='Missing KPI Count', ascending=False)

In [ ]:
len(df_x_1['Country Name'].unique()), len(selected_kpi_list)

In [ ]:
len(df_country_kpi_pop[df_country_kpi_pop['Missing KPI Count'] > 50].sort_values(by='Missing KPI Count', ascending=False))

In [ ]:
country_with_sporadic_data = df_country_kpi_pop[df_country_kpi_pop['Missing KPI Count'] > 50].sort_values(by='Missing KPI Count', ascending=False)['Country Name'].unique().tolist()
country_with_sporadic_data

In [ ]:
(df_x_1 == 0).any()

In [ ]:
df_x_2 = df_x_1[~df_x_1['Country Name'].isin(country_with_sporadic_data)]
df_x_2.shape

In [ ]:
df_x_2.to_csv('health_data_processed_2.csv', index=False)

In [ ]:
df_x_2[selected_kpi_list].isna().sum()

In [ ]:
df_x_2[['Country Name', 'Year', 'SP.DYN.AMRT.MA']]

In [ ]:
df_x_2.head()

In [ ]:
orig_df_x_1 = copy.deepcopy(df_x_1)
orig_df_x_2 = copy.deepcopy(df_x_2)

In [ ]:
orig_df_x_2.isna().any()

# Plots

In [ ]:
plot_kpi(df_x_2, 'Year', 'SP.POP.AG25.MA.IN', hue_col='Country Name')

In [ ]:
'''
for series in selected_kpi_list:
    print(series)
    plt.figure(figsize=(8, 8))
    sns.lineplot(data=df_x_1, x='Year', y=series, hue = 'Country Name')
    plt.legend(loc='lower center', bbox_to_anchor=(0.5, 1.0), ncol=3)
    plt.xticks(rotation=45, ha='right')
    # Add hover functionality
    cursor = mplcursors.cursor(hover=mplcursors.HoverMode.Transient)
    @cursor.connect("add")
    def on_add(sel):
        pass  # Do something with the selection
    plt.show()
    #plt.pause(5)
    #plt.close()
    break
'''

# Modeling

In [ ]:
for item in kpi_dict.items():
    if 'Life' in item[1]:
        print(item)

In [ ]:
'SP.DYN.LE00.IN' in df_x_2.columns, 'SP.DYN.LE00.MA.IN' in df_x_2.columns, 'SP.DYN.LE00.FE.IN' in df_x_2.columns

As Life expectancy at birth, total (years) is combination of Life expectancy at birth, female (years) and Life expectancy at birth, male (years), we are dropping these two columns Life expectancy at birth, female (years) and Life expectancy at birth, male (years).

In [ ]:
df_x_2.drop('SP.DYN.LE00.MA.IN', axis=1, inplace=True)
df_x_2.drop('SP.DYN.LE00.FE.IN', axis=1, inplace=True)

In [ ]:
df_x_2['Year'] = df_x_2['Year'].astype('int')

In [ ]:
dict(sorted(kpi_dict.items()))

In [ ]:
gender_specific_kpi_list = []
for k in selected_kpi_list:
    if 'female' in kpi_dict[k].lower() or 'male' in kpi_dict[k].lower():
        gender_specific_kpi_list.append(k)
len(gender_specific_kpi_list), len(selected_kpi_list)

In [ ]:
revised_kpi_list = [x for x in selected_kpi_list if x not in gender_specific_kpi_list]
len(revised_kpi_list)

In [ ]:
df_x_3 = df_x_2[['Country Name', 'Year'] + revised_kpi_list]
df_x_3.shape

In [ ]:
df_x_3.to_csv('health_data_processed_3.csv', index=False)

In [ ]:
df_x_3['Country Code'] = df_x_3['Country Name'].astype('category').cat.codes
country_dict = dict(zip(df_x_3['Country Code'], df_x_3['Country Name']))
df_x_3.drop('Country Name', axis=1, inplace=True)

In [ ]:
df_x_3.head()

In [ ]:
target_variable = 'SP.DYN.LE00.IN'

In [ ]:
y = df_x_3[target_variable]
X = df_x_3.drop(target_variable, axis=1)

In [ ]:
df_x_3.isna().any()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=X['Country Code'])

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Random Forest Regressor

In [ ]:
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
# Get feature importances from the trained model
importances_rf = rf_model.feature_importances_

# Instead of using 'selected_kpi_list', get the feature names directly from X_train
# This ensures the feature names match the importances array
feature_names = X_train.columns

# Create the DataFrame using the correct feature names
df_feature_importance_rf = pd.DataFrame({'Feature Code': feature_names, 'Gini Importance': importances_rf}).sort_values('Gini Importance', ascending=False)
df_feature_importance_rf['Feature Name'] = df_feature_importance_rf['Feature Code'].map(kpi_dict)
df_feature_importance_rf['Model'] = 'Random Forest Regressor'
df_feature_importance_rf

In [ ]:
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'Mean Squared Error (MSE): {mse_rf}')

In [ ]:
df_test_and_predicted_rf, df_actual_vs_predicted_rf = compare_actual_vs_predicted(X_test, y_test, y_pred_rf, 'Random Forest Regressor')
df_actual_vs_predicted_rf

In [ ]:
df_test_and_predicted_rf[['Country Code', 'Country Name', 'Year', 'Actual', 'Predicted']]

In [ ]:
plot_kpi(df_test_and_predicted_rf, 'Year', 'Actual', hue_col='Country Code')
plot_kpi(df_test_and_predicted_rf, 'Year', 'Predicted', hue_col='Country Code')

In [ ]:
plot_results(df_test_and_predicted_rf, 'Random Forest')

## XGBoost Regressor

In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.1, max_depth=3)
for col in X_train.select_dtypes(include=['object']).columns:
    try:
        X_train[col] = pd.to_numeric(X_train[col])
        X_test[col] = pd.to_numeric(X_test[col]) # Apply the same transformation to X_test
    except ValueError:
        print(f"Could not convert column '{col}' to numeric. You might need to handle it differently.")

# Now you can fit the model:
xgb_model.fit(X_train, y_train)
# Make predictions
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the model's performance
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print(f'Mean Squared Error: {mse_xgb}')

In [ ]:
# Get feature importances from the trained model
importances_xgb = xgb_model.feature_importances_

# Instead of using 'selected_kpi_list', get the feature names directly from X_train
# This ensures the feature names match the importances array
feature_names = X_train.columns

# Create the DataFrame using the correct feature names
df_feature_importance_xgb = pd.DataFrame({'Feature Code': feature_names, 'Gini Importance': importances_xgb}).sort_values('Gini Importance', ascending=False)
df_feature_importance_xgb['Feature Name'] = df_feature_importance_xgb['Feature Code'].map(kpi_dict)
df_feature_importance_xgb['Model'] = 'XGBoost Regressor'
df_feature_importance_xgb

In [ ]:
df_test_and_predicted_xgb, df_actual_vs_predicted_xgb = compare_actual_vs_predicted(X_test, y_test, y_pred_xgb, 'XGBoost Regressor')
df_actual_vs_predicted_xgb

In [ ]:
df_test_and_predicted_xgb[['Country Code', 'Country Name', 'Year', 'Actual', 'Predicted']]

In [ ]:
plot_kpi(df_test_and_predicted_xgb, 'Year', 'Actual', hue_col='Country Code')
plot_kpi(df_test_and_predicted_xgb, 'Year', 'Predicted', hue_col='Country Code')

In [ ]:
plot_results(df_test_and_predicted_xgb, 'XGBoost')

##LightGBM Regressor

In [ ]:
# Initialize the LightGBM model
lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.05, max_depth=3)

le = LabelEncoder()
y_train_1 = y_train.to_string()
y_test_1 = y_test.to_string()

#y_train['SP.DYN.LE00.IN'] = le.fit_transform(y_train['SP.DYN.LE00.IN'].astype(str))
#y_test['SP.DYN.LE00.IN'] = le.transform(y_test['SP.DYN.LE00.IN'].astype(str))

# After applying one of the above options, retrain the model:
lgb_model.fit(X_train, y_train)

# Make predictions
y_pred_lgbm = lgb_model.predict(X_test)

# Evaluate the model's performance
mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
print(f'Mean Squared Error: {mse_lgbm}')

In [ ]:
# Get feature importances from the trained model
importances_lgb = lgb_model.feature_importances_

# Instead of using 'selected_kpi_list', get the feature names directly from X_train
# This ensures the feature names match the importances array
feature_names = X_train.columns

# Create the DataFrame using the correct feature names
df_feature_importance_lgb = pd.DataFrame({'Feature Code': feature_names, 'Gini Importance': importances_lgb}).sort_values('Gini Importance', ascending=False)
df_feature_importance_lgb['Feature Name'] = df_feature_importance_lgb['Feature Code'].map(kpi_dict)
df_feature_importance_lgb['Model'] = 'LightGBM Regressor'
df_feature_importance_lgb

In [ ]:
df_test_and_predicted_lgbm, df_actual_vs_predicted_lgbm = compare_actual_vs_predicted(X_test, y_test, y_pred_lgbm, 'LightGBM Regressor')
df_actual_vs_predicted_lgbm

In [ ]:
df_test_and_predicted_lgbm[['Country Code', 'Country Name', 'Year', 'Actual', 'Predicted']]

In [ ]:
plot_kpi(df_test_and_predicted_lgbm, 'Year', 'Actual', hue_col='Country Code')
plot_kpi(df_test_and_predicted_lgbm, 'Year', 'Predicted', hue_col='Country Code')

In [ ]:
plot_results(df_test_and_predicted_xgb, 'LightGBM')

##Catboost Regressor

In [ ]:
# Initialize the CatBoost model
catboost_model = CatBoostRegressor(iterations=1000,
                                   learning_rate=0.1,
                                   depth=6,
                                   verbose=200)

# Train the model
catboost_model.fit(X_train, y_train)
# Make predictions
y_pred_catboost = catboost_model.predict(X_test)

# Evaluate the model's performance
mse_catboost = mean_squared_error(y_test, y_pred_catboost)
print(f'Mean Squared Error: {mse_catboost}')

In [ ]:
# Get feature importances from the trained model
importances_catboost = catboost_model.feature_importances_

# Instead of using 'selected_kpi_list', get the feature names directly from X_train
# This ensures the feature names match the importances array
feature_names = X_train.columns

# Create the DataFrame using the correct feature names
df_feature_importance_catboost = pd.DataFrame({'Feature Code': feature_names, 'Gini Importance': importances_catboost}).sort_values('Gini Importance', ascending=False)
df_feature_importance_catboost['Feature Name'] = df_feature_importance_catboost['Feature Code'].map(kpi_dict)
df_feature_importance_catboost['Model'] = 'CatBoost Regressor'
df_feature_importance_catboost

In [ ]:
df_test_and_predicted_catboost, df_actual_vs_predicted_catboost = compare_actual_vs_predicted(X_test, y_test, y_pred_catboost, 'CatBoost Regressor')
df_test_and_predicted_catboost

In [ ]:
df_test_and_predicted_catboost[['Country Code', 'Country Name', 'Year', 'Actual', 'Predicted']]

In [ ]:
plot_kpi(df_test_and_predicted_catboost, 'Year', 'Actual', hue_col='Country Code')
plot_kpi(df_test_and_predicted_catboost, 'Year', 'Predicted', hue_col='Country Code')

In [ ]:
plot_results(df_test_and_predicted_xgb, 'CatBoost')

##SVM Regressor

In [ ]:
# Initialize the Support Vector Regressor
svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

# Train the model
svr.fit(X_train, y_train)

# Predict on the test set
y_pred_svr = svr.predict(X_test)

# Evaluate the model
mse_svr = mean_squared_error(y_test, y_pred_svr)

print(f'Mean Squared Error: {mse_svr}')

It really performed so bad and predicted constant value across.

In [ ]:
df_test_and_predicted_svr, df_actual_vs_predicted_svr = compare_actual_vs_predicted(X_test, y_test, y_pred_svr, 'Support Vector Regressor')
df_test_and_predicted_svr

In [ ]:
df_test_and_predicted_svr[['Country Code', 'Country Name', 'Year', 'Actual', 'Predicted']]

##Panel Data Regression

In [ ]:
target_variable in df_x_4.columns, target_variable

In [ ]:
df_x_3.columns

In [ ]:
df_x_4 = copy.deepcopy(orig_df_x_2)
df_x_4.set_index(['Country Name', 'Year'], inplace=True)
df_x_4.head()

In [ ]:
# Fixed effects model
model_fe = PanelOLS.from_formula('SP.DYN.LE00.IN ~ SP.POP.TOTL + SP.POP.GROW + SP.URB.TOTL.IN.ZS + SP.URB.TOTL + SP.URB.GROW + SP.RUR.TOTL.ZS + SP.RUR.TOTL + SP.RUR.TOTL.ZG + SP.POP.65UP.TO.ZS + SP.POP.0014.TO.ZS + SP.POP.1564.TO.ZS + SP.POP.DPND + SP.POP.DPND.OL + SP.POP.DPND.YG + SP.POP.1564.TO + SP.POP.0014.TO + SP.POP.65UP.TO + SP.DYN.CBRT.IN + SP.DYN.CDRT.IN + SP.DYN.TFRT.IN + SH.DYN.MORT + SP.DYN.IMRT.IN + SH.DTH.MORT + SH.DTH.IMRT + SP.ADO.TFRT + Intercept + EntityEffects', data=df_x_4)
results_fe = model_fe.fit()

# Random effects model
model_re = PanelOLS.from_formula('SP.DYN.LE00.IN ~ SP.POP.TOTL + SP.POP.GROW + SP.URB.TOTL.IN.ZS + SP.URB.TOTL + SP.URB.GROW + SP.RUR.TOTL.ZS + SP.RUR.TOTL + SP.RUR.TOTL.ZG + SP.POP.65UP.TO.ZS + SP.POP.0014.TO.ZS + SP.POP.1564.TO.ZS + SP.POP.DPND + SP.POP.DPND.OL + SP.POP.DPND.YG + SP.POP.1564.TO + SP.POP.0014.TO + SP.POP.65UP.TO + SP.DYN.CBRT.IN + SP.DYN.CDRT.IN + SP.DYN.TFRT.IN + SH.DYN.MORT + SP.DYN.IMRT.IN + SH.DTH.MORT + SH.DTH.IMRT + SP.ADO.TFRT + Intercept + RandomEffects', data=df_x_4)
results_re = model_re.fit()

model_with_time_effects = PanelOLS.from_formula('SP.DYN.LE00.IN ~ SP.POP.TOTL + SP.POP.GROW + SP.URB.TOTL.IN.ZS + SP.URB.TOTL + SP.URB.GROW + SP.RUR.TOTL.ZS + SP.RUR.TOTL + SP.RUR.TOTL.ZG + SP.POP.65UP.TO.ZS + SP.POP.0014.TO.ZS + SP.POP.1564.TO.ZS + SP.POP.DPND + SP.POP.DPND.OL + SP.POP.DPND.YG + SP.POP.1564.TO + SP.POP.0014.TO + SP.POP.65UP.TO + SP.DYN.CBRT.IN + SP.DYN.CDRT.IN + SP.DYN.TFRT.IN + SH.DYN.MORT + SP.DYN.IMRT.IN + SH.DTH.MORT + SH.DTH.IMRT + SP.ADO.TFRT + Intercept + TimeEffects', data=df_x_4)
results_with_time_effects = model_with_time_effects.fit()
print(results_with_time_effects.summary())

# Hausman test
hausman_test_1 = lm.compare(results_fe, results_re)
print(hausman_test_1.summary())
hausman_test_2 = lm.compare(results_fe, results_with_time_effects)
print(hausman_test_2.summary())

In [ ]:
# Fixed Effects Model (controlling for individual-specific effects)
df_x_4['Intercept'] = 1
formula = 'SP.DYN.LE00.IN ~ SP.POP.TOTL + SP.POP.GROW + SP.URB.TOTL.IN.ZS + SP.URB.TOTL + SP.URB.GROW + SP.RUR.TOTL.ZS + SP.RUR.TOTL + SP.RUR.TOTL.ZG + SP.POP.65UP.TO.ZS + SP.POP.0014.TO.ZS + SP.POP.1564.TO.ZS + SP.POP.DPND + SP.POP.DPND.OL + SP.POP.DPND.YG + SP.POP.1564.TO + SP.POP.0014.TO + SP.POP.65UP.TO + SP.DYN.CBRT.IN + SP.DYN.CDRT.IN + SP.DYN.TFRT.IN + SH.DYN.MORT + SP.DYN.IMRT.IN + SH.DTH.MORT + SH.DTH.IMRT + SP.ADO.TFRT + EntityEffects + TimeEffects'
model_fixed = PanelOLS.from_formula('SP.DYN.LE00.IN ~ SP.POP.TOTL + SP.POP.GROW + SP.URB.TOTL.IN.ZS + SP.URB.TOTL + SP.URB.GROW + SP.RUR.TOTL.ZS + SP.RUR.TOTL + SP.RUR.TOTL.ZG + SP.POP.65UP.TO.ZS + SP.POP.0014.TO.ZS + SP.POP.1564.TO.ZS + SP.POP.DPND + SP.POP.DPND.OL + SP.POP.DPND.YG + SP.POP.1564.TO + SP.POP.0014.TO + SP.POP.65UP.TO + SP.DYN.CBRT.IN + SP.DYN.CDRT.IN + SP.DYN.TFRT.IN + SH.DYN.MORT + SP.DYN.IMRT.IN + SH.DTH.MORT + SH.DTH.IMRT + SP.ADO.TFRT', data=df_x_4)
results_fixed = model_fixed.fit()
print(results_fixed.summary)

'''
# Random Effects Model
model_random = PanelOLS.from_formula('SP.DYN.LE00.IN ~ SP.POP.TOTL + SP.POP.GROW + SP.URB.TOTL.IN.ZS + SP.URB.TOTL + SP.URB.GROW + SP.RUR.TOTL.ZS + SP.RUR.TOTL + SP.RUR.TOTL.ZG + SP.POP.65UP.TO.ZS + SP.POP.0014.TO.ZS + SP.POP.1564.TO.ZS + SP.POP.DPND + SP.POP.DPND.OL + SP.POP.DPND.YG + SP.POP.1564.TO + SP.POP.0014.TO + SP.POP.65UP.TO + SP.DYN.CBRT.IN + SP.DYN.CDRT.IN + SP.DYN.TFRT.IN + SH.DYN.MORT + SP.DYN.IMRT.IN + SH.DTH.MORT + SH.DTH.IMRT + SP.ADO.TFRT', data=df_x_3)
results_random = model_random.fit()
print(results_random.summary())

model_with_time_effects = PanelOLS.from_formula('SP.DYN.LE00.IN ~ SP.POP.TOTL + SP.POP.GROW + SP.URB.TOTL.IN.ZS + SP.URB.TOTL + SP.URB.GROW + SP.RUR.TOTL.ZS + SP.RUR.TOTL + SP.RUR.TOTL.ZG + SP.POP.65UP.TO.ZS + SP.POP.0014.TO.ZS + SP.POP.1564.TO.ZS + SP.POP.DPND + SP.POP.DPND.OL + SP.POP.DPND.YG + SP.POP.1564.TO + SP.POP.0014.TO + SP.POP.65UP.TO + SP.DYN.CBRT.IN + SP.DYN.CDRT.IN + SP.DYN.TFRT.IN + SH.DYN.MORT + SP.DYN.IMRT.IN + SH.DTH.MORT + SH.DTH.IMRT + SP.ADO.TFRT', data=df_x_3)
results_with_time_effects = model_with_time_effects.fit()
print(results_with_time_effects.summary())
'''

# Observations



1.   Dataframe is wide.
2.   Not all year-country-indicator column is populated.
3.   Country Name includes both individual countries and conglomerates. Ex. Arab World.



# Considerations



1.   Consider to shift timeline
2.   Check the KPI column population per country to decide, consider a threshold where for a particular KPI, if x countries have populated data which is above the threshold, consider it.



# Research Questions

Which features affect "Life expectancy at birth, total (years)" across different regions and socio economic conditions ('High income', 'Middle income', 'Low & middle income', 'Lower middle income', 'Low income')?

# Exploratory Data Analysis - Matt

In [ ]:
pd.set_option('display.max_columns', None)
# health_df[health_df['Indicator Name'] == 'Literacy rate, youth total (% of people ages 15-24)'].info()

In [ ]:
income_range_df = health_df[health_df['Country Name'].isin(['High income', 'Low & middle income', ' Lower middle income', 'Low income', 'Middle income'])]
display(income_range_df.shape, income_range_df.head())

In [ ]:
ed_indicators = ['Comprehensive correct knowledge of HIV/AIDS, ages 15-24, female (2 prevent ways and reject 3 misconceptions)',
                'Comprehensive correct knowledge of HIV/AIDS, ages 15-24, male (2 prevent ways and reject 3 misconceptions)',
                'Literacy rate, adult female (% of females ages 15 and above)',
                'Literacy rate, adult male (% of males ages 15 and above)',
                'Literacy rate, adult total (% of people ages 15 and above)',
                'Literacy rate, youth male (% of males ages 15-24)',
                'Literacy rate, youth total (% of people ages 15-24)',
                'Primary completion rate, female (% of relevant age group)',
                'Primary completion rate, male (% of relevant age group)',
                'Primary completion rate, total (% of relevant age group)',
                'Public spending on education, total (% of GDP)',
                'Ratio of school attendance of orphans to school attendance of non-orphans ages 10-14',
                'Ratio of young literate females to males (% ages 15-24)',
                'School enrollment, primary (% gross)',
                'School enrollment, primary (% net)',
                'School enrollment, primary, female (% gross)',
                'School enrollment, primary, female (% net)',
                'School enrollment, primary, male (% gross)',
                'School enrollment, primary, male (% net)',
                'School enrollment, secondary (% gross)',
                'School enrollment, secondary (% net)',
                'School enrollment, secondary, female (% gross)',
                'School enrollment, secondary, female (% net)',
                'School enrollment, secondary, male (% gross)',
                'School enrollment, secondary, male (% net)',
                'School enrollment, tertiary (% gross)',
                'School enrollment, tertiary, female (% gross)']

In [ ]:
# set number of rows to display
pd.set_option('display.max_rows', 150)
# drop all columns prior to 1970
columns_to_drop = [str(year) for year in range(1960, 1970)]
# income_range_df = income_range_df.drop(columns=columns_to_drop)
ed_df = income_range_df[income_range_df['Indicator Name'].isin(ed_indicators)].drop(columns=['Country Code', 'Indicator Code'])
ed_df
li_ed_df = ed_df[ed_df['Country Name'] == 'Low income'].drop(columns=['2014', '2015', 'Unnamed: 60'])
li_ed_df

In [ ]:
li_lit_df = li_ed_df[li_ed_df['Indicator Name'].isin([
    'Literacy rate, adult female (% of females ages 15 and above)',
    'Literacy rate, adult male (% of males ages 15 and above)',
    'Literacy rate, adult total (% of people ages 15 and above)',
    'Literacy rate, youth male (% of males ages 15-24)',
    'Literacy rate, youth total (% of people ages 15-24)',
    'Literacy rate, youth female (% of females ages 15-24)'
])].dropna(axis=1).drop(columns='Country Name').set_index('Indicator Name')
li_lit_df

In [ ]:
df = li_lit_df.transpose()
df.plot(kind='line', figsize=(10, 4))
plt.xlabel('Year')
plt.ylabel('Literacy Rate (%)')
plt.title('Literacy Rates Over Time')
plt.legend(title='Indicator Name', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

Let's look at formal education

In [ ]:
primary_completion_df = li_ed_df[li_ed_df['Indicator Name'].isin([
    'Primary completion rate, female (% of relevant age group)',
    'Primary completion rate, male (% of relevant age group)',
    'Primary completion rate, total (% of relevant age group)'
])].dropna(axis=1).drop(columns='Country Name').set_index('Indicator Name')
# primary_completion_df

In [ ]:
df = primary_completion_df.transpose()
df.plot(kind='line', figsize=(10, 4))
plt.title('Primary Completion Rates Over Time')
plt.xlabel('Year')
plt.ylabel('Primary Completion Rate (%)')
plt.legend(title='Indicator Name', bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
li_gni_df = health_df[health_df['Country Name'] == 'Low income']
li_gni_df = li_gni_df[li_gni_df['Indicator Name'] == 'GNI per capita, Atlas method (current US$)']
columns_to_drop = [str(year) for year in range(1960, 1988)]
li_gni_df = li_gni_df.drop(columns=['Country Code', 'Indicator Code', 'Country Name', 'Unnamed: 60']).drop(columns=columns_to_drop)
li_gni_df.set_index('Indicator Name', inplace=True)
# li_gni_df

In [ ]:
li_gni_df = li_gni_df.transpose()
li_gni_df.plot(kind='line', figsize=(10, 4))
plt.title('GNI per Capita Over Time')
plt.xlabel('Year')
plt.ylabel('GNI per Capita')


In [ ]:
health_df[health_df['Indicator Name'] == 'GNI per capita, Atlas method (current US$)'].describe()

In [ ]:
health_indicators = ['Prevalence of anemia among children (% of children under 5)',
       'Prevalence of anemia among non-pregnant women (% of women ages 15-49)',
       'Prevalence of anemia among pregnant women (%)',
       'Prevalence of anemia among women of reproductive age (% of women ages 15-49)',
       'Prevalence of HIV, female (% ages 15-24)',
       'Prevalence of HIV, male (% ages 15-24)',
       'Prevalence of HIV, total (% of population ages 15-49)',
       'Prevalence of overweight (% of adults)',
       'Prevalence of overweight (% of children under 5)',
       'Prevalence of overweight, female (% of children under 5)',
       'Prevalence of overweight, female (% of female adults)',
       'Prevalence of overweight, male (% of children under 5)',
       'Prevalence of overweight, male (% of male adults)',
       'Prevalence of severe wasting, weight for height (% of children under 5)',
       'Prevalence of severe wasting, weight for height, female (% of children under 5)',
       'Prevalence of severe wasting, weight for height, male (% of children under 5)',
       'Prevalence of syphilis (% of women attending antenatal care)',
       'Prevalence of tuberculosis (per 100,000 population)',
       'Prevalence of undernourishment (% of population)',
       'Prevalence of wasting (% of children under 5)',
       'Prevalence of wasting, female (% of children under 5)',
       'Prevalence of wasting, male (% of children under 5)',
       'Malnutrition prevalence, height for age (% of children under 5)',
       'Malnutrition prevalence, height for age, female (% of children under 5)',
       'Malnutrition prevalence, height for age, male (% of children under 5)',
       'Malnutrition prevalence, weight for age (% of children under 5)',
       'Malnutrition prevalence, weight for age, female (% of children under 5)',
       'Malnutrition prevalence, weight for age, male (% of children under 5)',
       'Maternal leave benefits (% of wages paid in covered period)',
       'Maternal mortality ratio (modeled estimate, per 100,000 live births)',
       'Maternal mortality ratio (national estimate, per 100,000 live births)',
       'Mortality caused by road traffic injury (per 100,000 people)',
       'Mortality rate, adult, female (per 1,000 female adults)',
       'Mortality rate, adult, male (per 1,000 male adults)',
       'Mortality rate, infant (per 1,000 live births)',
       'Mortality rate, infant, female (per 1,000 live births)',
       'Mortality rate, infant, male (per 1,000 live births)',
       'Mortality rate, neonatal (per 1,000 live births)',
       'Mortality rate, under-5 (per 1,000)',
       'Mortality rate, under-5, female (per 1,000)',
       'Mortality rate, under-5, male (per 1,000)']

In [ ]:
li_health_df = health_df[health_df['Indicator Name'].isin(health_indicators)][health_df['Country Name'] == 'Low income'].drop(columns=['Country Code', 'Indicator Code', 'Country Name', 'Unnamed: 60'])
li_health_df

In [ ]:
# li_health_df.isna().sum()

In [ ]:
li_adult_mortality_df = li_health_df[li_health_df['Indicator Name'].isin(['Mortality rate, adult, female (per 1,000 female adults)',
                                                                          'Mortality rate, adult, male (per 1,000 male adults)'])]
li_adult_mortality_df



In [ ]:
li_adult_mortality_df = li_adult_mortality_df.set_index('Indicator Name').transpose()



In [ ]:
# rename index to "Year"
li_adult_mortality_df.index.name = 'Year'
li_adult_mortality_df

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 4))
# add more space between plots
plt.subplots_adjust(hspace=0.5)

# plot adult mortality rate
axes[0][0].plot(li_adult_mortality_df.index, li_adult_mortality_df)
axes[0][0].set_title('Adult Mortality Rate by Year')
# only display xticks for every five years
axes[0][0].set_xticks(range(0, len(li_adult_mortality_df.index), 5))
axes[0][0].set_xticklabels(li_adult_mortality_df.index[::5])

# plot primary school completion rate
axes[0][1].plot(primary_completion_df.transpose().index, primary_completion_df.transpose())
axes[0][1].set_title('Primary School Completion Rate by Year')
axes[0][1].set_xticks(range(0, len(primary_completion_df.transpose().index), 5))
axes[0][1].set_xticklabels(primary_completion_df.transpose().index[::5])

# plot GNI per capita by year
axes[1][0].plot(li_gni_df.index, li_gni_df)
axes[1][0].set_title('GNI per Capita by Year')
axes[1][0].set_xticks(range(0, len(li_gni_df.index), 5))
axes[1][0].set_xticklabels(li_gni_df.index[::5])

# plot literacy rates over time
axes[1][1].plot(li_lit_df.transpose().index, li_lit_df.transpose())
axes[1][1].set_title('Literacy Rates by Year')


# Conclusions So Far
My hypothesis was that increasing education levels in low income countries might have a positive effect on life expectancy (or, as seen in the plots above, a decreasing effect on mortality rates). It seems clear, however, that at least in a broad sense, any effects of education are overshadowed by general wealth levels. That is to say, there's a definite correlation between education levels (as measured by primary school completion and literacy rates) and a decrease in adulkt mortality rates. However, GNI per capita seems to correlate even better with mortality rates than the chosen education metrics. (Note the dip and then plateau in GNI matches the brief rise in mortality rate from roughly 1990 to 2000.) This means we can't tell if education itself has anything to do with decreasing adult mortality rates, or if increased education and lower mortality rates are both effects of increasing wealth. Alternatively, it could be the case that education is the root cause of both decreasing mortality and increasing wealth.

As far as predictions go, this is not necessarily a problem. We could build a model to predict mortality rate that takes these factors (and others) into consideration. When it comes to elucidating causes and suggesting interventions, however, we want to know what has the most impact.

# Next Direction to Pursue
## Dig Deeper into the metrics
I suspect that the data I've examined so far will prove fairly representative, but I want to take a look at similar indicators to see if they follow the same patterns. I'll look at education indicators and health indicators in different categories - adult, youth - and in more specific areas, such HIV rates (which may correlate with sexual health education) and malaria incidence.

In [ ]:
reproductive_ed_indicators = [
    '% of females ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions)',
    '% of males ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions)'
    'Comprehensive correct knowledge of HIV/AIDS, ages 15-24, female (2 prevent ways and reject 3 misconceptions',
    'Comprehensive correct knowledge of HIV/AIDS, ages 15-24, male (2 prevent ways and reject 3 misconceptions)',
    'Adolescent fertility rate (births per 1,000 women ages 15-19)',
    'Condom use with non regular partner, % adults(15-49), female',
    'Condom use with non regular partner, % adults(15-49), male',
    'Condom use, population ages 15-24, female (% of females ages 15-24)',
    'Condom use, population ages 15-24, male (% of males ages 15-24)',
    'Contraceptive prevalence, any methods (% of women ages 15-49)',
    'Contraceptive prevalence, modern methods (% of women ages 15-49)'
    ]
hiv_outcomes = [
    'Children (0-14) living with HIV',
    'Children (ages 0-14) newly infected with HIV',
    'Children orphaned by HIV/AIDS',
    'Prevalence of HIV, female (% ages 15-24)',
    'Prevalence of HIV, male (% ages 15-24)',
    'Prevalence of HIV, total (% of population ages 15-49)',
    'Adults (ages 15+) and children (0-14 years) living with HIV',
    'Adults (ages 15+) and children (ages 0-14) newly infected with HIV',
    'Adults (ages 15+) living with HIV',
    'Adults (ages 15+) newly infected with HIV'
    ]

In [ ]:
low_income_df = health_df[health_df['Country Name'] == 'Low income']
reproductive_ed_df = low_income_df[low_income_df['Indicator Name'].isin(reproductive_ed_indicators)].drop(columns=['Country Code', 'Indicator Code', 'Country Name', 'Unnamed: 60'])
reproductive_ed_df

In [ ]:
reproductive_ed_df = reproductive_ed_df.dropna(axis=0, thresh=2)

In [ ]:
reproductive_ed_df

In [ ]:
reproductive_ed_df.set_index('Indicator Name').transpose().fillna(method='ffill').plot(kind='line', figsize=(10, 4))
plt.title('Reproductive Health Over Time')
plt.xlabel('Year')
plt.ylabel('Percentage')

In [ ]:
hiv_df = low_income_df[low_income_df['Indicator Name'].isin(hiv_outcomes)].drop(columns=['Country Code', 'Indicator Code', 'Country Name', 'Unnamed: 60'])
hiv_df

In [ ]:
hiv_df = hiv_df.set_index('Indicator Name').dropna(thresh=5).transpose().fillna(method='ffill')
hiv_df.plot(kind='line', figsize=(10, 4))
plt.title('HIV Outcomes Over Time')
plt.xlabel('Year')
plt.ylabel('Percentage')

# Reproductive Health
Here we see that higher contraceptive use, regardless of the specific method, is correlated with lower adolescent birth rates and lower prevalence of HIV over time.

Let's also look at:
* malaria
* spending on education
* education beyond primary
* separate countries that fall into low income level



In [ ]:
health_df[health_df['Indicator Name'] == 'GNI per capita, Atlas method (current US$)'].describe()

In [ ]:
gni_df = health_df[health_df['Indicator Name'] == 'GNI per capita, Atlas method (current US$)'].drop(columns=['Country Code', 'Indicator Code', 'Indicator Name', 'Unnamed: 60'])
gni_df

In [ ]:
# let's plot gni_df
gni_df.set_index('Country Name').transpose().plot(kind='line', figsize=(10, 4))
plt.title('GNI per Capita Over Time')
# no legend
plt.legend().set_visible(False)

That's not very informative...

# Let's Look at Individual Low-Income Countries
We'll create a threshold, based on an average of each country's GNI per capita over time, and examine the data and trends for all countries under that threshold.

In [ ]:
# let's find average gni for every Country Name in gni_df
gni_df[gni_df['Country Name'] == 'Low income']

In [ ]:
avg_gni = gni_df.bfill(axis=1)
avg_gni = avg_gni.set_index('Country Name')
avg_gni = avg_gni.ffill(axis=1).sort_values(by='Country Name')
avg_gni = avg_gni.ffill(axis=1).dropna()
avg_gni = avg_gni.mean(axis=1)
avg_gni.describe()

In [ ]:
avg_gni

I can use the info above to create my own list of low income countries and see if anything interesting shows up in the numbers for that group that is hidden in the aggregate view where country name = 'Low income'. Let's use the average value for the 'Low income' Country Name as our threshold, but let's bump it up a little bit to include a few more countries.

In [ ]:
avg_gni['Low income']

In [ ]:
threshhold = avg_gni['Low income'] + 150

In [ ]:
# build a custom list of low income countries
low_income_countries = avg_gni[avg_gni < threshhold].index.tolist()
low_income_countries.remove('Least developed countries: UN classification')  # remove the aggregate names
low_income_countries.remove('Low income')
low_income_countries.remove('Heavily indebted poor countries (HIPC)')
low_income_countries

In [ ]:
# filter the main df to include only those countries in the new list
new_df = health_df[health_df['Country Name'].isin(low_income_countries)]
new_df

In [ ]:
# build a df of education indicators for the low income countries
new_ed_df = new_df[new_df['Indicator Name'].isin(ed_indicators)]
# new_ed_df

In [ ]:
# new_ed_df.info()
# there's no non-null data in the columns 1960-1969, or in "unnamed", so let's drop those
new_ed_df = new_ed_df.drop(columns=['Country Code', 'Indicator Code']).dropna(axis=1, how='all')
# new_ed_df

In [ ]:
female_youth_hiv_ed_df = new_ed_df[new_ed_df['Indicator Name'] == 'Comprehensive correct knowledge of HIV/AIDS, ages 15-24, female (2 prevent ways and reject 3 misconceptions)'][['Country Name', '2005', '2006', '2010', '2014']].dropna(thresh=2)
# female_youth_hiv_ed_df
# 2005, 2006, 2010, 2014

In [ ]:
male_youth_hiv_ed_df = new_ed_df[new_ed_df['Indicator Name'] == 'Comprehensive correct knowledge of HIV/AIDS, ages 15-24, male (2 prevent ways and reject 3 misconceptions)'][['Country Name', '2005', '2006', '2010', '2014']].dropna(thresh=2)
# male_youth_hiv_ed_df

In [ ]:
youth_hiv_ed_df = male_youth_hiv_ed_df.merge(female_youth_hiv_ed_df, how='outer', on="Country Name", suffixes=("_male", "_female"))
youth_hiv_ed_df

In [ ]:
big_hiv_df = new_df[new_df['Indicator Name'].isin(['Prevalence of HIV, female (% ages 15-24)', 'Prevalence of HIV, male (% ages 15-24)'])].dropna(thresh=5).drop(columns=['Country Code', 'Indicator Code']).dropna(axis=1, thresh=5)[['Country Name', '2005', '2006', '2010', '2014']].merge(youth_hiv_ed_df, how='inner', on="Country Name")

In [ ]:
df = new_df[new_df['Indicator Name'] == 'Prevalence of HIV, female (% ages 15-24)'][['Country Name', '2005', '2006', '2010', '2014']].merge(female_youth_hiv_ed_df, how='inner', on="Country Name", suffixes=('_hiv', '_ed')).dropna(axis=1, thresh=3)

In [ ]:
import pandas as pd

# Assuming your DataFrame is named df
df_melted_hiv = df.melt(id_vars=['Country Name'], value_vars=['2005_hiv', '2006_hiv', '2010_hiv', '2014_hiv'],
                        var_name='Year', value_name='HIV')
df_melted_hiv['Year'] = df_melted_hiv['Year'].str.extract('(\d+)')

df_melted_ed = df.melt(id_vars=['Country Name'], value_vars=['2005_ed', '2006_ed', '2010_ed', '2014_ed'],
                       var_name='Year', value_name='Education')
df_melted_ed['Year'] = df_melted_ed['Year'].str.extract('(\d+)')

# Merge the two melted DataFrames
df_combined = pd.merge(df_melted_hiv, df_melted_ed, on=['Country Name', 'Year'])


In [ ]:
df_combined_cleaned = df_combined.dropna(subset=['HIV', 'Education'])

df_combined_cleaned

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Scatterplot with all countries combined
plt.figure(figsize=(12, 8))
# sns.scatterplot(data=df_combined_cleaned, x="Education", y="HIV", hue="Country Name", palette="tab20", alpha=0.8)
sns.regplot(data=df_combined_cleaned, x="Education", y="HIV", scatter=True)

# Add labels and title
plt.title("Correlation Between Education and HIV Prevalence Across Countries", fontsize=16)
plt.xlabel("HIV Education Level (%)", fontsize=14)
plt.ylabel("HIV Prevalence (%)", fontsize=14)
# plt.legend(title="Country", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()



In [ ]:
adult_lit = new_df[new_df['Indicator Name'] == 'Literacy rate, adult total (% of people ages 15 and above)']
adult_lit

In [ ]:
ed_indicators